In [148]:
import findspark
findspark.init("/opt/spark/")
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import regexp_replace
import configparser
import os
from pyspark.sql import functions as F
from pyspark.sql.functions import month, date_format

In [149]:
spark = SparkSession.builder \
.appName("Odev_2") \
.config("spark.jars.packages", "org.postgresql:postgresql:42.7.2") \
.master("local[2]") \
.getOrCreate()

In [3]:
! java --version


openjdk 11.0.22 2024-01-16 LTS
OpenJDK Runtime Environment (Red_Hat-11.0.22.0.7-1.el7_9) (build 11.0.22+7-LTS)
OpenJDK 64-Bit Server VM (Red_Hat-11.0.22.0.7-1.el7_9) (build 11.0.22+7-LTS, mixed mode, sharing)


In [ ]:
#her bir csv dosyası için dataframe olusturma

In [4]:
data_dir = '/opt/examples/datasets/retail_db'
csv_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]

In [192]:
dfs = []
for csv_file in csv_files:
    df = spark.read.option("header", True).option("inferSchema", True).csv(csv_file)
    dfs.append(df)

categories_df = dfs[0]
customers_df = dfs[1]
departments_df = dfs[2]
orders_df = dfs[3]
order_items_df = dfs[4]
products_df = dfs[5]


In [193]:
#createOrReplaceTempView

order_items_df.createOrReplaceTempView("order_items")
orders_df.createOrReplaceTempView("orders")
products_df.createOrReplaceTempView("products")
categories_df.createOrReplaceTempView("categories")
customers_df.createOrReplaceTempView("customers")
departments_df.createOrReplaceTempView("departments")


In [194]:
### 1. How many distinct `orderItemOrderId` in `order_items`

#1. `Order_ıtems' içinde kaç farklı `orderItemOrderId` var

spark.sql(f"""

SELECT COUNT(DISTINCT orderItemOrderId) AS distinct_orderItemOrderId_count
FROM order_items;

""").toPandas()

,distinct_orderItemOrderId_count
0,57431


In [9]:
### 2. Find the row counts of `orders` and `order_items` tables.

### 2. `Siparişler` ve `order_ıtems` tablolarının satır sayılarını bulun.


#orders

spark.sql(
    f"""
    select count(*) AS orders_row_count from orders 



""").toPandas()


#order_items

spark.sql(
    f"""
    select count(*) AS order_items_row_count from order_items



""").toPandas()







,order_items_row_count
0,172198


In [ ]:
### 3. Write the most canceled (descending order) products in terms of total sales amount to the local disk in 'parquet' format.


### 3. Toplam satış tutarı açısından en çok iptal edilen (azalan sipariş) ürünleri 'parke' formatında yerel diske yazın.

closed_products_df=spark.sql(
    f"""

    SELECT p.productName, sum(od.orderItemSubTotal) AS total_sales_amount
    FROM order_items od
    INNER JOIN orders o ON o.orderId = od.orderItemOrderId
    INNER JOIN products p ON p.productId = od.orderItemProductId
    WHERE o.orderStatus = 'CLOSED'
    GROUP BY p.productName
    ORDER BY total_sales_amount desc
    




""")

closed_products_df.write.parquet("/opt/examples/datasets/retail_db/closed_products")





In [ ]:
#parquet dosyasını okuma

In [37]:
df_parquet = spark.read.parquet("/opt/examples/datasets/retail_db/closed_products")
df_parquet.toPandas()

,productName,total_sales_amount
0,Field & Stream Sportsman 16 Gun Fire Safe,764361.78
1,Perfect Fitness Perfect Rip Deck,468041.98
2,Diamondback Women's Serene Classic Comfort Bi,447870.14
3,Nike Men's Free 5.0+ Running Shoe,421657.83
4,Nike Men's Dri-FIT Victory Golf Polo,340400.00
...,...,...
94,Nike Men's Fingertrap Max Training Shoe,499.96
95,GoPro HERO3+ Black Edition Camera,399.99
96,Nike Men's Kobe IX Elite Low Basketball Shoe,399.98
97,Stiga Master Series ST3100 Competition Indoor,329.99


In [ ]:
### 4. Write the most canceled (descending order) categories in terms of total sales amount to the local disk in 'parquet' format.

### 4. Toplam satış tutarı açısından en çok iptal edilen (azalan sipariş) kategorileri 'parke' formatında yerel diske yazın.

closed_categoires_df=spark.sql(
    f"""

    SELECT c.categoryName,sum(od.orderItemSubTotal) AS total_sales_amount
    FROM order_items od
    INNER JOIN orders o ON o.orderId = od.orderItemOrderId
    INNER JOIN products p ON p.productId = od.orderItemProductId
    inner join categories c on c.categoryId=p.productCategoryId
    WHERE o.orderStatus = 'CLOSED'
    GROUP BY c.categoryName
    ORDER BY total_sales_amount desc
    




""")
    
closed_categoires_df.write.parquet("/opt/examples/datasets/retail_db/closed_categories")

In [12]:
df_parquet_categories=spark.read.parquet("/opt/examples/datasets/retail_db/closed_categories")
df_parquet_categories.toPandas()

,categoryName,total_sales_amount
0,Fishing,764361.78
1,Cleats,468641.96
2,Camping & Hiking,447870.14
3,Cardio Equipment,424327.83
4,Water Sports,341032.97
5,Women's Apparel,340400.00
6,Men's Footwear,321075.30
7,Indoor/Outdoor Games,309676.08
8,Shop By Sport,137176.55
9,Electronics,38074.38


In [27]:
### 5. In which month of which year (Turkish) was the highest total sales?

### 5. Toplam satışlar hangi yılın hangi ayında (Türkçe) en yüksekti?
orders_df.dtypes


[('orderId', 'int'),
 ('orderDate', 'timestamp'),
 ('orderCustomerId', 'int'),
 ('orderStatus', 'string')]

In [28]:

#orderDate sutunudaki degerlere göre ayın sayısını yeni bir sutunda gosterme

orders_df_2 = orders_df.withColumn("orderDate_month", 
                     F.month(F.col("orderDate")))

orders_df_2.show()

+-------+-------------------+---------------+---------------+---------------+
|orderId|          orderDate|orderCustomerId|    orderStatus|orderDate_month|
+-------+-------------------+---------------+---------------+---------------+
|      1|2013-07-25 00:00:00|          11599|         CLOSED|              7|
|      2|2013-07-25 00:00:00|            256|PENDING_PAYMENT|              7|
|      3|2013-07-25 00:00:00|          12111|       COMPLETE|              7|
|      4|2013-07-25 00:00:00|           8827|         CLOSED|              7|
|      5|2013-07-25 00:00:00|          11318|       COMPLETE|              7|
|      6|2013-07-25 00:00:00|           7130|       COMPLETE|              7|
|      7|2013-07-25 00:00:00|           4530|       COMPLETE|              7|
|      8|2013-07-25 00:00:00|           2911|     PROCESSING|              7|
|      9|2013-07-25 00:00:00|           5657|PENDING_PAYMENT|              7|
|     10|2013-07-25 00:00:00|           5648|PENDING_PAYMENT|   

In [34]:

#türkçe ay adları için fonksiyon:

def month_tr(month_num):
    
    months_dict={
        1:"Ocak",
        2:"Şubat",
        3:"Mart",
        4:"Nisan",
        5:"Mayıs",
        6:"Haziran",
        7:"Temmuz",
        8:"Ağustos",
        9:"Eylül",
        10:"Ekim",
        11:"Kasım",
        12:"Aralık"
    }
    
    return months_dict[month_num]

#fonksiyonu udf kaydetme
month_tr_udf=F.udf(month_tr,StringType())
    

In [35]:
#türkçe ay adları ıcın fonksiyonu calıstırarak yenı bır sutun olusturma


orders_df_2=orders_df_2.withColumn("orderDate_Month_Name",
                              month_tr_udf(F.col("orderDate_Month")))

    

In [36]:
orders_df_2.show()

+-------+-------------------+---------------+---------------+---------------+--------------------+
|orderId|          orderDate|orderCustomerId|    orderStatus|orderDate_month|orderDate_Month_Name|
+-------+-------------------+---------------+---------------+---------------+--------------------+
|      1|2013-07-25 00:00:00|          11599|         CLOSED|              7|              Temmuz|
|      2|2013-07-25 00:00:00|            256|PENDING_PAYMENT|              7|              Temmuz|
|      3|2013-07-25 00:00:00|          12111|       COMPLETE|              7|              Temmuz|
|      4|2013-07-25 00:00:00|           8827|         CLOSED|              7|              Temmuz|
|      5|2013-07-25 00:00:00|          11318|       COMPLETE|              7|              Temmuz|
|      6|2013-07-25 00:00:00|           7130|       COMPLETE|              7|              Temmuz|
|      7|2013-07-25 00:00:00|           4530|       COMPLETE|              7|              Temmuz|
|      8|2

In [42]:

orders_df_2.createOrReplaceTempView("orders_2")

In [74]:

#sql sorgusu

spark.sql(
    f"""
    
    SELECT Year,
           Month,
           TotalSales
    FROM (
        SELECT YEAR(orderDate) AS Year,
               orderDate_Month_Name AS Month,
               SUM(orderItemSubTotal) AS TotalSales,
               ROW_NUMBER() OVER(PARTITION BY YEAR(orderDate) ORDER BY SUM(orderItemSubTotal) DESC) AS rn
        FROM orders_2 o 
        INNER JOIN order_items od ON od.orderItemOrderId = o.orderId
        WHERE YEAR(orderDate) IN (2013, 2014)
        GROUP BY YEAR(orderDate), orderDate_Month_Name
    ) tmp
    WHERE rn = 1
  

""").toPandas()
    

,Year,Month,TotalSales
0,2013,Kasım,3168656.03
1,2014,Ocak,2924447.01


In [104]:
### 6. On which day of the week (Turkish) was the highest total sales?
### 6. Haftanın hangi günü (Türkçe) en yüksek toplam satış oldu?



#orderDate sutunudaki degerlere göre haftayı yeni bir sutunda gosterme

orders_df_3 = orders_df.withColumn("orderDate_weekofyear", 
                     F.weekofyear(F.col("orderDate")))


#orderDate sutunudaki degerlere göre günü yeni bir satıtrda gösterme
orders_df_4 = orders_df_3.withColumn("orderDate_dayofweek", 
                     F.dayofweek(F.col("orderDate")))

orders_df_4.show()


+-------+-------------------+---------------+---------------+--------------------+-------------------+
|orderId|          orderDate|orderCustomerId|    orderStatus|orderDate_weekofyear|orderDate_dayofweek|
+-------+-------------------+---------------+---------------+--------------------+-------------------+
|      1|2013-07-25 00:00:00|          11599|         CLOSED|                  30|                  5|
|      2|2013-07-25 00:00:00|            256|PENDING_PAYMENT|                  30|                  5|
|      3|2013-07-25 00:00:00|          12111|       COMPLETE|                  30|                  5|
|      4|2013-07-25 00:00:00|           8827|         CLOSED|                  30|                  5|
|      5|2013-07-25 00:00:00|          11318|       COMPLETE|                  30|                  5|
|      6|2013-07-25 00:00:00|           7130|       COMPLETE|                  30|                  5|
|      7|2013-07-25 00:00:00|           4530|       COMPLETE|            

In [112]:
#dayname(eng)

orders_df_5 = orders_df_4.withColumn("dayname_eng",
                                     F.date_format(F.col("orderDate"),"E"))

orders_df_5.show()


+-------+-------------------+---------------+---------------+--------------------+-------------------+-----------+
|orderId|          orderDate|orderCustomerId|    orderStatus|orderDate_weekofyear|orderDate_dayofweek|dayname_eng|
+-------+-------------------+---------------+---------------+--------------------+-------------------+-----------+
|      1|2013-07-25 00:00:00|          11599|         CLOSED|                  30|                  5|        Thu|
|      2|2013-07-25 00:00:00|            256|PENDING_PAYMENT|                  30|                  5|        Thu|
|      3|2013-07-25 00:00:00|          12111|       COMPLETE|                  30|                  5|        Thu|
|      4|2013-07-25 00:00:00|           8827|         CLOSED|                  30|                  5|        Thu|
|      5|2013-07-25 00:00:00|          11318|       COMPLETE|                  30|                  5|        Thu|
|      6|2013-07-25 00:00:00|           7130|       COMPLETE|                  3

In [113]:

#gün isimlerini türkçe olarak tabloya ekleme

def day_tr(day_num):
    
    day_dict={
        1:"Pazar",
        2:"Pazartesi",
        3:"Salı",
        4:"Çarşamba",
        5:"Perşembe",
        6:"Cuma",
        7:"Cumartesi",
    }
    
    return day_dict[day_num]

#fonksiyonu udf kaydetme
day_tr_udf=F.udf(day_tr,StringType())



In [114]:
#türkçe gün adları ıcın fonksiyonu calıstırarak yenı bır sutun olusturma


orders_df_6=orders_df_5.withColumn("orderDate_day_tr",
                              day_tr_udf(F.col("orderDate_dayofweek")))

orders_df_6.show()


+-------+-------------------+---------------+---------------+--------------------+-------------------+-----------+----------------+
|orderId|          orderDate|orderCustomerId|    orderStatus|orderDate_weekofyear|orderDate_dayofweek|dayname_eng|orderDate_day_tr|
+-------+-------------------+---------------+---------------+--------------------+-------------------+-----------+----------------+
|      1|2013-07-25 00:00:00|          11599|         CLOSED|                  30|                  5|        Thu|        Perşembe|
|      2|2013-07-25 00:00:00|            256|PENDING_PAYMENT|                  30|                  5|        Thu|        Perşembe|
|      3|2013-07-25 00:00:00|          12111|       COMPLETE|                  30|                  5|        Thu|        Perşembe|
|      4|2013-07-25 00:00:00|           8827|         CLOSED|                  30|                  5|        Thu|        Perşembe|
|      5|2013-07-25 00:00:00|          11318|       COMPLETE|               

In [118]:
orders_df_6.createOrReplaceTempView("orders_6")

In [145]:
#sql sorgusu

spark.sql(
    f"""
  SELECT Year,Week,Day,TotalSales
    FROM (
        SELECT YEAR(orderDate) AS Year,
               orderDate_dayofweek AS Week,
                orderDate_day_tr AS Day,
               SUM(orderItemSubTotal) AS TotalSales,
               ROW_NUMBER() OVER(PARTITION BY YEAR(orderDate) ORDER BY SUM(orderItemSubTotal) DESC) AS rn
        FROM orders_6 o 
        INNER JOIN order_items od ON od.orderItemOrderId = o.orderId
        WHERE YEAR(orderDate) IN (2013, 2014)
        GROUP BY YEAR(orderDate), orderDate_dayofweek,orderDate_day_tr
    ) 
    WHERE rn = 1

""").toPandas()



  



  

,Year,Week,Day,TotalSales
0,2013,5,Perşembe,2302285.55
1,2014,6,Cuma,2895567.33


In [146]:
### 7. Create the largest table possible from all these tables and write it to a table named `retail_all` in the postgresql `test1` database.
### 7. Tüm bu tablolardan mümkün olan en büyük tabloyu oluşturun ve postgresql `test1` veritabanındaki `retail_all` adlı bir tabloya yazın.


In [202]:
 spark.sql(
    f"""
  SELECT 
        od.*,
        o.*,
        p.*,
        c.*,
        cs.*,
        d.*
    FROM order_items od
    FULL OUTER JOIN orders o 
    on o.orderId=od.orderItemOrderId
    FULL OUTER JOIN products p
    on p.productId=od.orderItemProductId
    FULL OUTER JOIN categories c
    on c.categoryId=p.productCategoryId
    FULL OUTER JOIN customers  cs
    on cs.customerId=o.orderCustomerId
    FULL OUTER JOIN departments  d
    on d.departmentId=c.categoryDepartmentId
  
    
    
    

""").toPandas()



TypeError: Casting to unit-less dtype 'datetime64' is not supported. Pass e.g. 'datetime64[ns]' instead.

In [ ]:
#2.yöntem

In [181]:
orders_with_items = order_items_df.join(orders_df, order_items_df.orderItemOrderId == orders_df.orderId, "outer")

In [182]:
retail_all_df = orders_with_items \
    .join(products_df, orders_with_items.orderItemProductId == products_df.productId, "outer") \
    .join(categories_df, products_df.productCategoryId == categories_df.categoryId, "outer") \
    .join(customers_df, orders_with_items.orderCustomerId == customers_df.customerId, "outer") \
    .join(departments_df, categories_df.categoryDepartmentId == departments_df.departmentId, "outer")

In [188]:
retail_all_df.show()

+-------------+----------------+------------------+-----------------+-----------------+---------------------+-------+---------+---------------+-----------+---------+-----------------+--------------------+------------------+------------+--------------------+----------+--------------------+------------+----------+-------------+-------------+-------------+----------------+--------------+------------+-------------+---------------+------------+--------------+
|orderItemName|orderItemOrderId|orderItemProductId|orderItemQuantity|orderItemSubTotal|orderItemProductPrice|orderId|orderDate|orderCustomerId|orderStatus|productId|productCategoryId|         productName|productDescription|productPrice|        productImage|categoryId|categoryDepartmentId|categoryName|customerId|customerFName|customerLName|customerEmail|customerPassword|customerStreet|customerCity|customerState|customerZipcode|departmentId|departmentName|
+-------------+----------------+------------------+-----------------+-------------

In [198]:
#postgresql

config = configparser.RawConfigParser()


config.read('./db_conn')
user_name = config.get('DB', 'user_name')
password = config.get('DB', 'password')
db_ip = config.get('DB', 'db_ip')



In [199]:
db_ip

'localhost'

In [200]:
jdbcUrl = f"jdbc:postgresql://{db_ip}:5433/test1?user={user_name}&password={password}"

In [201]:
retail_all_df.write.jdbc(url=jdbcUrl,
              table="retail_all", 
              mode="overwrite", 
              properties={"driver": 'org.postgresql.Driver'})